In [1]:
#from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

import pandas as pd
import numpy as np
import random
import tensorflow as tf

from gensim.models import Word2Vec

In [2]:
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences
from keras_preprocessing.text import Tokenizer
from keras_preprocessing.sequence import pad_sequences

from tensorflow.keras.optimizers import RMSprop, Adam
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding,Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Sequential 
#from tensorflow.keras.models import Sequential, load_model
#from tensorflow.keras.layers import LSTM, Dense, Dropout, Embedding, Masking, Bidirectional
#from tensorflow.keras.optimizers import Adam
#from tensorflow.keras import backend

In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import LancasterStemmer

nltk.download('omw-1.4')
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [4]:
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [5]:
data = pd.read_csv("/content/drive/My Drive/NLP/Project/NLPMiniProject_data.csv")
print(data.head())

           ID                                            comment     date  \
0  uid_590555  Well, let's be honest here, they don't actuall...  2015-04   
1  uid_671762  Well, I didn't need evidence to believe in com...  2016-12   
2  uid_519689              Who does an "official promo" in 360p?  2013-11   
3  uid_788362                           Grotto koth was the best  2015-09   
4  uid_299252                                   Neal's back baby  2015-11   

   down                                     parent_comment  score  top  \
0     0  They should shut the fuck up and let the commu...      2    2   
1    -1  You need evidence to kill people? I thought we...      6   -1   
2     0                    2014 BMW S1000R: Official Promo      3    3   
3     0  Not really that memorable lol if you want memo...      2    2   
4     0                      James Neal hit on Zach Parise     -5   -5   

              topic            user  label  
0       starcitizen  Combat_Wombatz      0  
1 

In [6]:
df = data[['ID','comment','parent_comment','down','score','top','topic','label']]

In [7]:
df['comment'][0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*."

In [8]:
df['parent_comment'][0]

'They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate.'

In [9]:
(df['comment']+df['parent_comment'])[0]

"Well, let's be honest here, they don't actually seem to do much moderating, so they have to spend their time doing *something*.They should shut the fuck up and let the community decide what it wants for the subreddit. They are here to **moder** fucking **ate**, not to dictate."

In [10]:
X_train, X_test, y_train, y_test = train_test_split(
    df['comment']+df['parent_comment'],
    df['label'],
    test_size=0.2, 
    random_state=42
)

In [11]:
X_train.shape, X_test.shape

((12000,), (3000,))

# Build the Tokenizer

In [12]:
desired_vocab_size = 10000 #Vocablury size
t = tf.keras.preprocessing.text.Tokenizer(num_words=desired_vocab_size, oov_token=32) # num_words -> Vocablury size

In [13]:
#Fit tokenizer with actual training data
t.fit_on_texts(X_train.tolist())

In [14]:
#Vocabulary
t.word_index

{32: 1,
 'the': 2,
 'to': 3,
 'a': 4,
 'and': 5,
 'i': 6,
 'of': 7,
 'is': 8,
 'you': 9,
 'that': 10,
 'in': 11,
 'it': 12,
 'for': 13,
 'this': 14,
 'on': 15,
 'be': 16,
 'but': 17,
 'with': 18,
 'have': 19,
 'not': 20,
 'are': 21,
 'they': 22,
 'was': 23,
 'just': 24,
 'so': 25,
 'my': 26,
 'if': 27,
 'like': 28,
 'he': 29,
 'what': 30,
 'as': 31,
 'all': 32,
 "it's": 33,
 'or': 34,
 'at': 35,
 'would': 36,
 'do': 37,
 'people': 38,
 'your': 39,
 'get': 40,
 'me': 41,
 'we': 42,
 'can': 43,
 "don't": 44,
 'about': 45,
 'no': 46,
 'from': 47,
 'one': 48,
 'because': 49,
 'an': 50,
 'out': 51,
 'up': 52,
 "i'm": 53,
 'how': 54,
 'will': 55,
 'there': 56,
 'more': 57,
 'when': 58,
 'who': 59,
 'them': 60,
 'has': 61,
 'their': 62,
 'think': 63,
 'know': 64,
 'his': 65,
 'by': 66,
 'only': 67,
 'some': 68,
 'time': 69,
 'good': 70,
 'really': 71,
 'why': 72,
 'game': 73,
 'than': 74,
 'well': 75,
 'even': 76,
 'now': 77,
 'then': 78,
 'should': 79,
 'had': 80,
 'him': 81,
 'been': 82,
 '

# Prepare Training and Test Data

Get the word index for each of the word in the review

In [15]:
X_train[0:1]

9839    "black shield", xpecial confirmed racist and i...
dtype: object

In [16]:
X_train = t.texts_to_sequences(X_train.tolist())

In [17]:
print(X_train[0:1])

[[295, 1630, 9908, 1232, 459, 5, 4922, 45, 65, 1, 171, 3646, 9908, 414, 729]]


In [18]:
X_test = t.texts_to_sequences(X_test)

# Pad Sequences - Important

In [19]:
#Define maximum number of words to consider in each review
max_review_length = 300

In [20]:
#Pad training and test reviews
X_train = tf.keras.preprocessing.sequence.pad_sequences(X_train,
                                                        maxlen=max_review_length,
                                                        padding='pre', 
                                                        truncating='post')

X_test = tf.keras.preprocessing.sequence.pad_sequences(X_test, 
                                                       maxlen=max_review_length, 
                                                       padding='pre',
                                                       truncating='post')

In [21]:
X_train.shape,X_test.shape

((12000, 300), (3000, 300))

In [22]:
X_train[100]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   

# Build the Graph

In [23]:
tf.random.set_seed(42)

In [24]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

Add Embedding layer
 - Embedding Layer Input = Batch_Size * Length of each review

In [25]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    input_length=max_review_length) #Number of words in each review
          )

In [26]:
model.input

<KerasTensor: shape=(None, 300) dtype=float32 (created by layer 'embedding_input')>

In [27]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

Embedding Layer Output - [Batch_Size , Review Length , Embedding_Size]

Flatten the Output

In [28]:
model.add(tf.keras.layers.Flatten())

In [29]:
model.output

<KerasTensor: shape=(None, 15000) dtype=float32 (created by layer 'flatten')>

Add Dense Layer

In [30]:
model.add(tf.keras.layers.BatchNormalization())

In [31]:
model.output

<KerasTensor: shape=(None, 15000) dtype=float32 (created by layer 'batch_normalization')>

In [32]:
model.add(tf.keras.layers.Dense(128, activation='relu'))

In [33]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dense')>

In [34]:
model.add(tf.keras.layers.Dropout(0.25))

In [35]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'dropout')>

Use Dense layer for output layer

In [36]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [37]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [38]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 batch_normalization (BatchN  (None, 15000)            60000     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               1920128   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

# Execute the graph

In [ ]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 12s 30ms/step - loss: 0.7161 - accuracy: 0.5284 - val_loss: 0.6902 - val_accuracy: 0.5470
Epoch 2/5
375/375 [==============================] - 11s 30ms/step - loss: 0.5014 - accuracy: 0.7513 - val_loss: 0.8710 - val_accuracy: 0.5560
Epoch 3/5
375/375 [==============================] - 13s 35ms/step - loss: 0.2114 - accuracy: 0.9173 - val_loss: 1.4055 - val_accuracy: 0.5473
Epoch 4/5
375/375 [==============================] - 12s 31ms/step - loss: 0.1112 - accuracy: 0.9620 - val_loss: 1.1563 - val_accuracy: 0.5470
Epoch 5/5
375/375 [==============================] - 11s 30ms/step - loss: 0.0795 - accuracy: 0.9754 - val_loss: 0.9851 - val_accuracy: 0.5580


Using Pre-Trained Embeddings

In [47]:
import csv
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchtext

In [48]:
# The first time you run this will download a ~823MB file
glove = torchtext.vocab.GloVe(name="6B", dim=50, max_vectors=20000)

.vector_cache/glove.6B.zip: 862MB [02:38, 5.43MB/s]                           
100%|█████████▉| 19999/20000 [00:00<00:00, 32047.31it/s]


In [49]:
glove.vectors.shape

torch.Size([20000, 50])

In [50]:
glove['india']

tensor([-0.2036, -0.8707, -0.1917,  0.7386,  0.1849,  0.1493,  0.4808, -0.2163,
         0.7275, -0.3691,  0.1340, -0.1143, -0.1807, -0.6468, -0.1848,  0.8357,
         0.4818,  0.7603, -0.5038,  0.8074,  1.2195,  0.3459,  0.2218,  0.3133,
         1.2066, -1.8441,  0.1406, -0.9972, -1.1402,  0.3234,  3.2128,  0.4271,
         0.1950,  0.8011,  0.3855, -0.1257, -0.2653,  0.0553, -1.1557,  0.1684,
        -0.8223,  0.2039,  0.0892, -0.6012, -0.0329,  1.3735, -0.5166,  0.2961,
         0.2395, -1.3801])

In [51]:
#Initialize embedding matrix for our dataset with 10000+1 rows (1 for padding word)
#and 50 columns (as embedding size is 50)
embedding_matrix = np.zeros((desired_vocab_size + 1, 50))

In [52]:
embedding_matrix.shape

(10001, 50)

In [53]:
embedding_matrix[200]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [54]:
for word, i in sorted(t.word_index.items(),key=lambda x:x[1]):
    if i > (desired_vocab_size+1):
        break
    try:
        embedding_vector = glove[word] #Reading word's embedding from Glove model for a given word
        embedding_matrix[i] = embedding_vector
    except:
        pass

In [55]:
embedding_matrix[200]

array([ 0.29262999,  0.31020001, -0.12961   ,  0.35857999,  0.59204   ,
        0.46941   , -0.024792  , -0.30658999,  0.20464   ,  0.77373999,
       -0.14436001,  0.42105001,  0.030358  ,  0.01449   ,  0.39918   ,
        0.38801   ,  0.098906  ,  0.39401001,  0.10773   , -0.95802999,
        0.041812  ,  0.044068  , -0.24857999,  0.0076151 ,  0.38519999,
       -1.63950002, -0.68910998,  0.43243   ,  0.93482   , -0.33768001,
        2.9611001 ,  0.15918   , -0.19574   ,  0.12766001,  0.21367   ,
       -0.2272    , -0.21646   ,  0.41707   , -0.023072  , -0.74263   ,
        0.28744999,  0.46162   ,  0.18258999,  0.019844  ,  0.18715   ,
        0.54868001,  0.24677999,  0.29536   , -0.0064379 ,  0.80203003])

Building a model using Pre-Trained embeddings

In [56]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [57]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [58]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [59]:
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dropout(0.25))
model.add(tf.keras.layers.Dense(1, activation='sigmoid'))

In [60]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [61]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 flatten (Flatten)           (None, 15000)             0         
                                                                 
 batch_normalization (BatchN  (None, 15000)            60000     
 ormalization)                                                   
                                                                 
 dense (Dense)               (None, 128)               1920128   
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 129       
                                                        

In [62]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 11s 26ms/step - loss: 1.0101 - accuracy: 0.5180 - val_loss: 0.7001 - val_accuracy: 0.5540
Epoch 2/5
375/375 [==============================] - 9s 25ms/step - loss: 0.7212 - accuracy: 0.6040 - val_loss: 0.8391 - val_accuracy: 0.5413
Epoch 3/5
375/375 [==============================] - 9s 25ms/step - loss: 0.6324 - accuracy: 0.6389 - val_loss: 0.8220 - val_accuracy: 0.5417
Epoch 4/5
375/375 [==============================] - 11s 28ms/step - loss: 0.5767 - accuracy: 0.6762 - val_loss: 0.8916 - val_accuracy: 0.5320
Epoch 5/5
375/375 [==============================] - 10s 26ms/step - loss: 0.5498 - accuracy: 0.7083 - val_loss: 1.0522 - val_accuracy: 0.5313


# With Simple RNN

In [63]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [64]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [65]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

Add Simple RNN layer

In [66]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.SimpleRNN(128, activation='relu')) #RNN State - size of memory

In [67]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'simple_rnn')>

In [68]:
model.add(tf.keras.layers.Dropout(0.25))

Output layer

In [69]:
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [70]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [71]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 simple_rnn (SimpleRNN)      (None, 128)               22912     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 523,291
Trainable params: 23,141
Non-trainable params: 500,150
_____________________________________________

In [72]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 37s 95ms/step - loss: 0.8973 - accuracy: 0.5009 - val_loss: 0.7029 - val_accuracy: 0.5160
Epoch 2/5
375/375 [==============================] - 34s 90ms/step - loss: 0.7126 - accuracy: 0.5270 - val_loss: 0.7010 - val_accuracy: 0.5117
Epoch 3/5
375/375 [==============================] - 36s 96ms/step - loss: 0.6900 - accuracy: 0.5403 - val_loss: 0.6999 - val_accuracy: 0.5083
Epoch 4/5
375/375 [==============================] - 34s 90ms/step - loss: 0.6763 - accuracy: 0.5676 - val_loss: 0.6992 - val_accuracy: 0.5203
Epoch 5/5
375/375 [==============================] - 36s 95ms/step - loss: 0.6668 - accuracy: 0.5838 - val_loss: 0.7044 - val_accuracy: 0.5150


In [73]:
model.fit(X_train,y_train,
          epochs=10,
          initial_epoch=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 6/10
375/375 [==============================] - 34s 90ms/step - loss: 0.6538 - accuracy: 0.5982 - val_loss: 0.7038 - val_accuracy: 0.5220
Epoch 7/10
375/375 [==============================] - 36s 96ms/step - loss: 0.6427 - accuracy: 0.6187 - val_loss: 0.7040 - val_accuracy: 0.5347
Epoch 8/10
375/375 [==============================] - 34s 90ms/step - loss: 0.6304 - accuracy: 0.6279 - val_loss: 0.7168 - val_accuracy: 0.5210
Epoch 9/10
375/375 [==============================] - 34s 91ms/step - loss: 0.6111 - accuracy: 0.6446 - val_loss: 0.7197 - val_accuracy: 0.5270
Epoch 10/10
375/375 [==============================] - 34s 91ms/step - loss: 0.5943 - accuracy: 0.6653 - val_loss: 0.7414 - val_accuracy: 0.5310


In [74]:
from sklearn.metrics import accuracy_score, recall_score, classification_report, cohen_kappa_score
from sklearn import metrics 

In [75]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 2s 24ms/step


In [76]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [77]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  53.1

 Classification Report:
               precision    recall  f1-score   support

           0       0.53      0.56      0.54      1493
           1       0.54      0.50      0.52      1507

    accuracy                           0.53      3000
   macro avg       0.53      0.53      0.53      3000
weighted avg       0.53      0.53      0.53      3000



# Using LSTM

In [78]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [79]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [80]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [81]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.LSTM(128)) #RNN State - size of memory

In [82]:
model.output

<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'lstm')>

In [83]:
model.add(tf.keras.layers.Dropout(0.25))

In [84]:
#Output
model.add(tf.keras.layers.Dense(1,activation='sigmoid'))

In [85]:
#Compile the model
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [86]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 lstm (LSTM)                 (None, 128)               91648     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                                 
 dense (Dense)               (None, 1)                 129       
                                                                 
Total params: 592,027
Trainable params: 91,877
Non-trainable params: 500,150
_____________________________________________

In [87]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 172s 454ms/step - loss: 0.6954 - accuracy: 0.5326 - val_loss: 0.6803 - val_accuracy: 0.5613
Epoch 2/5
375/375 [==============================] - 163s 434ms/step - loss: 0.6587 - accuracy: 0.6083 - val_loss: 0.6745 - val_accuracy: 0.5930
Epoch 3/5
375/375 [==============================] - 158s 420ms/step - loss: 0.6281 - accuracy: 0.6446 - val_loss: 0.6773 - val_accuracy: 0.5870
Epoch 4/5
375/375 [==============================] - 161s 429ms/step - loss: 0.5878 - accuracy: 0.6886 - val_loss: 0.6864 - val_accuracy: 0.5903
Epoch 5/5
375/375 [==============================] - 158s 421ms/step - loss: 0.5323 - accuracy: 0.7372 - val_loss: 0.7029 - val_accuracy: 0.5983


In [88]:
model.fit(X_train,y_train,
          epochs=10,
          initial_epoch=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 6/10
375/375 [==============================] - 158s 423ms/step - loss: 0.4722 - accuracy: 0.7758 - val_loss: 0.7591 - val_accuracy: 0.6007
Epoch 7/10
375/375 [==============================] - 159s 424ms/step - loss: 0.3945 - accuracy: 0.8288 - val_loss: 0.8281 - val_accuracy: 0.5950
Epoch 8/10
375/375 [==============================] - 158s 423ms/step - loss: 0.3312 - accuracy: 0.8595 - val_loss: 0.9038 - val_accuracy: 0.5840
Epoch 9/10
375/375 [==============================] - 159s 423ms/step - loss: 0.2656 - accuracy: 0.8935 - val_loss: 1.0106 - val_accuracy: 0.5803
Epoch 10/10
375/375 [==============================] - 159s 425ms/step - loss: 0.2099 - accuracy: 0.9186 - val_loss: 1.1007 - val_accuracy: 0.5830


In [89]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 13s 137ms/step


In [90]:
predictions1 = [0 if x < 0.5 else 1 for x in predictions]

In [91]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  58.3

 Classification Report:
               precision    recall  f1-score   support

           0       0.58      0.58      0.58      1493
           1       0.59      0.58      0.58      1507

    accuracy                           0.58      3000
   macro avg       0.58      0.58      0.58      3000
weighted avg       0.58      0.58      0.58      3000



# Using BiLSTM

In [92]:
#Initialize model
tf.keras.backend.clear_session()
model = tf.keras.Sequential()

In [93]:
model.add(tf.keras.layers.Embedding(desired_vocab_size + 1, #Vocablury size
                                    50, #Embedding size
                                    weights=[embedding_matrix],
                                    trainable=False,
                                    input_length=max_review_length) #Number of words in each review
          )

In [94]:
model.output

<KerasTensor: shape=(None, 300, 50) dtype=float32 (created by layer 'embedding')>

In [95]:
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(200, return_sequences=True), merge_mode='sum'))

In [96]:
model.output

<KerasTensor: shape=(None, 300, 200) dtype=float32 (created by layer 'bidirectional')>

In [97]:
model.add(Dropout(0.5))
model.add(Dense(1,activation = 'sigmoid'))

In [98]:
model.compile(loss = 'binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [99]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 300, 50)           500050    
                                                                 
 batch_normalization (BatchN  (None, 300, 50)          200       
 ormalization)                                                   
                                                                 
 bidirectional (Bidirectiona  (None, 300, 200)         401600    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 300, 200)          0         
                                                                 
 dense (Dense)               (None, 300, 1)            201       
                                                                 
Total params: 902,051
Trainable params: 401,901
Non-trai

In [100]:
model.fit(X_train,y_train,
          epochs=5,
          batch_size=32,          
          validation_data=(X_test, y_test))

Epoch 1/5
375/375 [==============================] - 471s 1s/step - loss: 0.6964 - accuracy: 0.4988 - val_loss: 0.6924 - val_accuracy: 0.5092
Epoch 2/5
375/375 [==============================] - 467s 1s/step - loss: 0.6921 - accuracy: 0.5120 - val_loss: 0.6912 - val_accuracy: 0.5093
Epoch 3/5
375/375 [==============================] - 462s 1s/step - loss: 0.6909 - accuracy: 0.5045 - val_loss: 0.6903 - val_accuracy: 0.5157
Epoch 4/5
375/375 [==============================] - 461s 1s/step - loss: 0.6890 - accuracy: 0.5154 - val_loss: 0.7008 - val_accuracy: 0.5068
Epoch 5/5
375/375 [==============================] - 464s 1s/step - loss: 0.6892 - accuracy: 0.5103 - val_loss: 0.6900 - val_accuracy: 0.5130


In [101]:
####### Prediction ##########
predictions = model.predict(X_test)

94/94 [==============================] - 33s 337ms/step


In [103]:
predictions1 = [0 if x.any() < 0.5 else 1 for x in predictions]

In [104]:
print('Baseline: Accuracy: ', round(accuracy_score(y_test, predictions1)*100, 2))
print('\n Classification Report:\n', classification_report(y_test,predictions1))

Baseline: Accuracy:  50.23

 Classification Report:
               precision    recall  f1-score   support

           0       0.00      0.00      0.00      1493
           1       0.50      1.00      0.67      1507

    accuracy                           0.50      3000
   macro avg       0.25      0.50      0.33      3000
weighted avg       0.25      0.50      0.34      3000



/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.7/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
